In [1]:
# Library Imports
import shutil
import os
import re
import cv2
import numpy as np
from six.moves import range

In [8]:
# Regular Expression for splitting by whitespace
splitter = re.compile("\s+")
base_path = './data/img/'

In [4]:
# Read the relevant annotation file and preprocess it
with open('./data/Eval/list_eval_partition.txt', 'r') as eval_partition_file:
    list_eval_partition = [line.rstrip('\n') for line in eval_partition_file][2:]
    list_eval_partition = [splitter.split(line) for line in list_eval_partition]
    list_all = [(v[0][4:], v[0].split('/')[1].split('_')[-1], v[1]) for v in list_eval_partition]

In [9]:
# Put each image into the relevant folder in train/test/validation folder
for element in list_all:
    if not os.path.exists(os.path.join(base_path, element[2])):
        os.mkdir(os.path.join(base_path, element[2]))
    if not os.path.exists(os.path.join(os.path.join(base_path, element[2]), element[1])):
        os.mkdir(os.path.join(os.path.join(base_path, element[2]), element[1]))
    if not os.path.exists(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                          element[0].split('/')[0])):
        os.mkdir(os.path.join(os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1])),
                 element[0].split('/')[0]))
    shutil.move(os.path.join(base_path, element[0]),
                os.path.join(os.path.join(os.path.join(base_path, element[2]), element[1]), element[0]))

FileNotFoundError: [Errno 2] No such file or directory: './data/img/Sheer_Pleated-Front_Blouse/img_00000001.jpg'